# Import dependencies

In [13]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [14]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from models.BERT_Dropout_BiLSTM_Linear import BERT_Dropout_BiLSTM_Linear

In [15]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [16]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [17]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [18]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [19]:
df = pd.json_normalize(json.load(open(DATASET)))

In [20]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [21]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [22]:
BERT_FINE_TUNED_PATH = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [23]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_bilstm_linear.csv'

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [25]:
best_accuracy = 0.0

In [26]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [27]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [28]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [29]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [30]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = BERT_Dropout_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.2310829162597656
Epoch: 0/2, Batch: 50/501, Loss: 0.29282650351524353
Epoch: 0/2, Batch: 100/501, Loss: 0.13377776741981506
Epoch: 0/2, Batch: 150/501, Loss: 0.2031979262828827
Epoch: 0/2, Batch: 200/501, Loss: 0.09002398699522018
Epoch: 0/2, Batch: 250/501, Loss: 0.09110508114099503
Epoch: 0/2, Batch: 300/501, Loss: 0.12059461325407028
Epoch: 0/2, Batch: 350/501, Loss: 0.0393509604036808
Epoch: 0/2, Batch: 400/501, Loss: 0.027842549607157707
Epoch: 0/2, Batch: 450/501, Loss: 0.09087565541267395
Epoch: 0/2, Batch: 500/501, Loss: 0.06600473821163177
Epoch: 1/2, Batch: 0/501, Loss: 0.11092810332775116
Epoch: 1/2, Batch: 50/501, Loss: 0.025190506130456924
Epoch: 1/2, Batch: 100/501, Loss: 0.06172860786318779
Epoch: 1/2, Batch: 150/501, Loss: 0.05311788618564606
Epoch: 1/2, Batch: 200/501, Loss: 0.03558126837015152
Epoch: 1/2, Batch: 250/501, Loss: 0.06435580551624298
Epoch: 1/2, Batch: 300/501, Loss: 0.01338998693972826
Epoch: 1/2, Batch: 350/501

In [31]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.991843,0.991843,0.964069,0.991843,0.971585,0.991843,0.967604,113.780996
1,0.992267,0.992267,0.956891,0.992267,0.977282,0.992267,0.966774,100.590002
2,0.988162,0.988162,0.958969,0.988162,0.913056,0.988162,0.934476,100.867003
3,0.983683,0.983683,0.895807,0.983683,0.963374,0.983683,0.927423,100.978001
4,0.990569,0.990569,0.943243,0.990569,0.966362,0.990569,0.954562,100.475029
5,0.987472,0.987472,0.930815,0.987472,0.953164,0.987472,0.940467,101.199029
6,0.990097,0.990097,0.939725,0.990097,0.968227,0.990097,0.953537,100.505996
7,0.986130,0.986130,0.919279,0.986130,0.971171,0.986130,0.943294,100.929030
8,0.990088,0.990088,0.944022,0.990088,0.962344,0.990088,0.953021,100.482998
9,0.988246,0.988246,0.920482,0.988246,0.964751,0.988246,0.941520,100.830030


In [32]:
results.to_csv(STATS_OUTPUT)